# 🏢 End-to-End Order & Complaint Management Agent

**Unified Customer Service Agent powered by Microsoft Agent Framework**

This notebook demonstrates a hybrid architecture combining:
- ✅ **Custom Python Tools** - Order Manager Library
- ✅ **MCP Server Tools** - Complaint Management Server
- ✅ **Multi-turn Conversations** - Stateful session management
- ✅ **Formal Customer Service** - Professional interaction tone

## Architecture Overview

```
┌─────────────────────────────────────────────────────┐
│         MAF Agent (Azure OpenAI)                    │
│  "Customer Service Representative"                  │
└──────────────┬──────────────────────┬───────────────┘
               │                      │
       ┌───────▼────────┐    ┌───────▼─────────────┐
       │ Custom Tools    │    │ MCP HTTP Tool       │
       │ (@tool wrapped) │    │ (External Server)   │
       └───────┬────────┘    └───────┬─────────────┘
               │                      │
       ┌───────▼────────┐    ┌───────▼─────────────┐
       │ OrderManager   │    │ FastMCP Server      │
       │ (SQLite)       │    │ (SQLite)            │
       └────────────────┘    └─────────────────────┘
     orders.db             complaints.db
```

## Customer Journey Scenario

This notebook simulates a realistic customer service interaction:
1. 📦 Customer creates an order
2. 🔍 Customer checks order status
3. 🚚 Order status updated to shipped
4. ⚠️ Customer reports delivery delay complaint
5. 🔎 Agent searches customer's orders and complaints
6. ✅ Agent resolves the complaint

In [3]:
# Cell 1: Imports and Setup
import os
import sys
from datetime import datetime
from typing import Annotated, Dict, List, Optional

from dotenv import load_dotenv
from pydantic import Field

from agent_framework import tool
from agent_framework.azure import AzureOpenAIResponsesClient
from agent_framework import MCPStreamableHTTPTool
from azure.identity.aio import AzureCliCredential

# Add order_manager library to path
# Use absolute path based on notebook location to ensure reliability
notebook_dir = os.path.dirname(os.path.abspath("__file__")) if "__file__" in dir() else os.getcwd()
# Navigate up from experiments/ to v15-e2e-casestudy/ then to libraries
order_manager_path = os.path.abspath(os.path.join(notebook_dir, '..', '..', 'back-end', 'libraries'))

print(f"📍 Notebook directory: {notebook_dir}")
print(f"📂 Order Manager path: {order_manager_path}")
print(f"✓ Path exists: {os.path.exists(order_manager_path)}")

if order_manager_path not in sys.path:
    sys.path.insert(0, order_manager_path)
    print(f"✓ Added to sys.path")

from order_manager import (
    OrderManager,
    Order,
    Config,
    OrderNotFoundException,
    InvalidOrderDataException,
    ValidationException
)

print("✅ All imports successful")

📍 Notebook directory: c:\000-MAGIC\v15\practices\magic-ai-practices\v15-e2e-casestudy\back-end\experiments
📂 Order Manager path: c:\000-MAGIC\v15\practices\magic-ai-practices\v15-e2e-casestudy\back-end\libraries
✓ Path exists: True
✓ Added to sys.path
✅ All imports successful


In [4]:
# Cell 2: Load Configuration
load_dotenv(override=True)

project_endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")
model = os.getenv("AZURE_OPENAI_RESPONSES_DEPLOYMENT_NAME")

print(f"Project Endpoint: {project_endpoint}")
print(f"Model: {model}")
print(f"✅ Configuration loaded")

Project Endpoint: https://magicv15foundry.services.ai.azure.com/api/projects/magicv15project
Model: gpt-4o
✅ Configuration loaded


In [5]:
# Cell 3: Initialize Order Manager
# Database path for orders (library manages this)
order_db_path = "back-end/data/orders_e2e.db"

# Create OrderManager instance
order_manager = OrderManager(db_path=order_db_path)

print(f"✅ OrderManager initialized with database: {order_db_path}")
print(f"📊 Valid Order Statuses: {', '.join(Config.VALID_STATUSES)}")

✅ OrderManager initialized with database: back-end/data/orders_e2e.db
📊 Valid Order Statuses: Pending, Processing, Confirmed, Shipped, Delivered, Cancelled, Returned


In [6]:
# Cell 4: Create Custom Tools from Order Manager

@tool(approval_mode="never_require")
def create_customer_order(
    customer_name: Annotated[str, Field(description="Full name of the customer")],
    billing_address: Annotated[str, Field(description="Customer's billing address")],
    product_sku: Annotated[str, Field(description="Product SKU/code")],
    quantity: Annotated[int, Field(description="Order quantity, must be positive")],
    order_amount: Annotated[int, Field(description="Total order amount in cents (e.g., 299900 for $2999.00)")],
    remarks: Annotated[Optional[str], Field(description="Optional notes or comments")] = None
) -> str:
    """
    Create a new customer order in the order management system.
    This tool creates orders with 'Pending' status by default.
    """
    try:
        order = order_manager.create_order(
            order_date=datetime.now(),
            customer_name=customer_name,
            billing_address=billing_address,
            product_sku=product_sku,
            quantity=quantity,
            order_amount=order_amount,
            order_status="Pending",
            remarks=remarks
        )
        return f"✅ Order created successfully.\n\nOrder Details:\n- Order ID: {order.order_id}\n- Customer: {order.customer_name}\n- Product: {order.product_sku}\n- Quantity: {order.quantity}\n- Amount: ${order.order_amount / 100:.2f}\n- Status: {order.order_status}\n- Order Date: {order.order_date.strftime('%Y-%m-%d %H:%M:%S')}"
    except (InvalidOrderDataException, ValidationException) as e:
        return f"❌ Order creation failed: {str(e)}"
    except Exception as e:
        return f"❌ An unexpected error occurred: {str(e)}"


@tool(approval_mode="never_require")
def get_customer_orders(
    customer_name: Annotated[str, Field(description="Exact customer name to search for")]
) -> str:
    """
    Retrieve all orders for a specific customer (exact name match).
    Returns complete order history for the customer.
    """
    try:
        orders = order_manager.get_orders_by_customer(customer_name)
        
        if not orders:
            return f"No orders found for customer: {customer_name}"
        
        result = f"📦 Found {len(orders)} order(s) for {customer_name}:\n\n"
        for order in orders:
            result += f"Order ID: {order.order_id}\n"
            result += f"  Product: {order.product_sku}\n"
            result += f"  Quantity: {order.quantity}\n"
            result += f"  Amount: ${order.order_amount / 100:.2f}\n"
            result += f"  Status: {order.order_status}\n"
            result += f"  Order Date: {order.order_date.strftime('%Y-%m-%d %H:%M:%S')}\n"
            if order.remarks:
                result += f"  Remarks: {order.remarks}\n"
            result += "\n"
        
        return result
    except Exception as e:
        return f"❌ Error retrieving orders: {str(e)}"


@tool(approval_mode="never_require")
def get_order_details(
    order_id: Annotated[int, Field(description="The order ID to retrieve")]
) -> str:
    """
    Get detailed information about a specific order by its ID.
    """
    try:
        order = order_manager.get_order_by_id(order_id)
        
        result = f"📋 Order Details (ID: {order.order_id}):\n\n"
        result += f"Customer: {order.customer_name}\n"
        result += f"Billing Address: {order.billing_address}\n"
        result += f"Product SKU: {order.product_sku}\n"
        result += f"Quantity: {order.quantity}\n"
        result += f"Amount: ${order.order_amount / 100:.2f}\n"
        result += f"Status: {order.order_status}\n"
        result += f"Order Date: {order.order_date.strftime('%Y-%m-%d %H:%M:%S')}\n"
        if order.remarks:
            result += f"Remarks: {order.remarks}\n"
        result += f"\nCreated: {order.created_at.strftime('%Y-%m-%d %H:%M:%S')}\n"
        result += f"Last Updated: {order.updated_at.strftime('%Y-%m-%d %H:%M:%S')}"
        
        return result
    except OrderNotFoundException as e:
        return f"❌ {str(e)}"
    except Exception as e:
        return f"❌ Error retrieving order: {str(e)}"


@tool(approval_mode="never_require")
def search_orders_by_customer_name(
    customer_name_partial: Annotated[str, Field(description="Partial customer name to search (case-insensitive)")]
) -> str:
    """
    Search for orders using partial customer name match.
    This is useful when you don't know the exact customer name.
    """
    try:
        orders = order_manager.search_orders_by_customer(customer_name_partial)
        
        if not orders:
            return f"No orders found matching: {customer_name_partial}"
        
        result = f"🔍 Found {len(orders)} order(s) matching '{customer_name_partial}':\n\n"
        for order in orders:
            result += f"Order ID: {order.order_id} | Customer: {order.customer_name} | Product: {order.product_sku} | Status: {order.order_status}\n"
        
        return result
    except Exception as e:
        return f"❌ Error searching orders: {str(e)}"


@tool(approval_mode="never_require")
def update_order_status(
    order_id: Annotated[int, Field(description="The order ID to update")],
    new_status: Annotated[str, Field(description="New status - must be one of: Pending, Processing, Confirmed, Shipped, Delivered, Cancelled, Returned")]
) -> str:
    """
    Update the status of an existing order.
    Valid statuses: Pending, Processing, Confirmed, Shipped, Delivered, Cancelled, Returned
    """
    try:
        order = order_manager.update_order_status(order_id, new_status)
        return f"✅ Order {order_id} status updated to: {order.order_status}\n\nUpdated at: {order.updated_at.strftime('%Y-%m-%d %H:%M:%S')}"
    except OrderNotFoundException as e:
        return f"❌ {str(e)}"
    except ValidationException as e:
        return f"❌ {str(e)}\n\nValid statuses: {', '.join(Config.VALID_STATUSES)}"
    except Exception as e:
        return f"❌ Error updating order: {str(e)}"


@tool(approval_mode="never_require")
def search_orders_advanced(
    order_status: Annotated[Optional[str], Field(description="Filter by order status")] = None,
    product_sku: Annotated[Optional[str], Field(description="Filter by product SKU")] = None,
    billing_address_partial: Annotated[Optional[str], Field(description="Filter by partial billing address")] = None
) -> str:
    """
    Advanced search for orders with multiple filter options.
    You can filter by status, product SKU, or billing address (partial match).
    Multiple filters are combined with AND logic.
    """
    try:
        orders = order_manager.search_orders(
            order_status=order_status,
            product_sku=product_sku,
            billing_address_partial=billing_address_partial
        )
        
        if not orders:
            return "No orders found matching the search criteria."
        
        filters = []
        if order_status:
            filters.append(f"Status: {order_status}")
        if product_sku:
            filters.append(f"SKU: {product_sku}")
        if billing_address_partial:
            filters.append(f"Address contains: {billing_address_partial}")
        
        result = f"🔍 Found {len(orders)} order(s) matching criteria ({', '.join(filters)}):\n\n"
        for order in orders:
            result += f"Order ID: {order.order_id} | Customer: {order.customer_name} | Product: {order.product_sku} | Status: {order.order_status} | Amount: ${order.order_amount / 100:.2f}\n"
        
        return result
    except Exception as e:
        return f"❌ Error searching orders: {str(e)}"


print("✅ Created 6 custom order management tools:")
print("   1. create_customer_order")
print("   2. get_customer_orders")
print("   3. get_order_details")
print("   4. search_orders_by_customer_name")
print("   5. update_order_status")
print("   6. search_orders_advanced")

✅ Created 6 custom order management tools:
   1. create_customer_order
   2. get_customer_orders
   3. get_order_details
   4. search_orders_by_customer_name
   5. update_order_status
   6. search_orders_advanced


In [7]:
# Cell 5: Setup MCP Tool for Complaint Management

complaint_mcp_tool = MCPStreamableHTTPTool(
    name="Complaint Management System",
    url="http://localhost:8000/mcp"
)

print("✅ MCP Tool configured for Complaint Management")
print(f"   Server URL: http://localhost:8000/mcp")
print(f"   Note: Ensure MCP server is running before executing agent queries")

✅ MCP Tool configured for Complaint Management
   Server URL: http://localhost:8000/mcp
   Note: Ensure MCP server is running before executing agent queries


In [8]:
# Cell 6: Create Unified Customer Service Agent

credential = AzureCliCredential()
client = AzureOpenAIResponsesClient(
    project_endpoint=project_endpoint,
    deployment_name=model,
    credential=credential,
)

# Create agent with both custom tools and MCP tool
agent = client.as_agent(
    name="Customer Service Representative",
    instructions="""You are a professional customer service representative for an e-commerce company.

Your responsibilities include:
- Managing customer orders (creation, tracking, status updates)
- Handling customer complaints professionally and empathetically
- Providing accurate information about order status and delivery
- Resolving issues efficiently while maintaining a formal, courteous tone

Guidelines:
1. Always address customers respectfully and professionally
2. Provide clear, detailed responses with all relevant information
3. Use proper formatting for order and complaint details
4. When customers report issues, first check their order history before creating complaints
5. Link complaints to specific order IDs when applicable
6. Keep customers informed about status changes
7. Use appropriate tools based on the customer's request

Available Tools:
- Order Management: create_customer_order, get_customer_orders, get_order_details, search_orders_by_customer_name, update_order_status, search_orders_advanced
- Complaint Management: create_complaint, get_complaint, update_complaint, list_complaints, filter_complaints, delete_complaint (via MCP server)

Always strive to provide excellent customer service and resolve issues promptly.""",
    tools=[
        # Custom Python Tools (Order Manager)
        create_customer_order,
        get_customer_orders,
        get_order_details,
        search_orders_by_customer_name,
        update_order_status,
        search_orders_advanced,
        # MCP Tool (Complaint Manager)
        complaint_mcp_tool
    ]
)

print("✅ Customer Service Agent created")
print(f"   Agent Name: {agent.name}")
print(f"   Total Tools: 6 custom tools + 1 MCP tool (6 complaint tools)")
print(f"   Instructions: Formal customer service tone")

✅ Customer Service Agent created
   Agent Name: Customer Service Representative
   Total Tools: 6 custom tools + 1 MCP tool (6 complaint tools)
   Instructions: Formal customer service tone


---

## 🎭 Customer Service Scenario

**Customer:** Ramkumar from Sydney, Australia  
**Journey Timeline:**
1. Places order for a laptop
2. Checks order status
3. Order shipped
4. Reports delivery delay issue
5. Agent investigates and resolves

Let's begin the multi-turn conversation...

In [10]:
# Cell 7: Start Customer Session

session = agent.create_session()
print("✅ Customer service session started")
print("\n" + "="*80)
print("CUSTOMER SERVICE INTERACTION LOG")
print("="*80 + "\n")

✅ Customer service session started

CUSTOMER SERVICE INTERACTION LOG



In [11]:
# Cell 8: Turn 1 - Customer Creates Order

print("🗣️ CUSTOMER: Turn 1")
print("-" * 80)

query_1 = """Hello, I would like to place an order for a laptop.

My details:
- Name: Ramkumar
- Billing Address: 123 George Street, Sydney, NSW 2000, Australia
- Product: LAPTOP-HP-001
- Quantity: 1
- Amount: $2,499.00

Please process this order for me."""

print(f"Query: {query_1}\n")

try:
    response_1 = await agent.run(query_1, session=session)
    print("🤖 AGENT RESPONSE:")
    print(response_1)
except Exception as e:
    print(f"❌ Error: {str(e)}")

print("\n" + "="*80 + "\n")

🗣️ CUSTOMER: Turn 1
--------------------------------------------------------------------------------
Query: Hello, I would like to place an order for a laptop.

My details:
- Name: Ramkumar
- Billing Address: 123 George Street, Sydney, NSW 2000, Australia
- Product: LAPTOP-HP-001
- Quantity: 1
- Amount: $2,499.00

Please process this order for me.

🤖 AGENT RESPONSE:
Hello Ramkumar,

Your order for the laptop has been successfully processed. Here are the details:

**Order Details:**
- **Order ID:** 1
- **Customer Name:** Ramkumar
- **Product SKU:** LAPTOP-HP-001
- **Quantity:** 1
- **Total Amount:** $2,499.00
- **Billing Address:** 123 George Street, Sydney, NSW 2000, Australia
- **Status:** Pending
- **Order Date:** 19th February 2026

Your order is currently in the "Pending" status. You will receive notifications as it progresses to the next stages. If you have any questions or need further assistance, please feel free to reach out.

Thank you for choosing our service!

Warm regards,


In [12]:
# Cell 9: Turn 2 - Customer Checks Order Status

print("🗣️ CUSTOMER: Turn 2")
print("-" * 80)

query_2 = """Can you please check the status of my recent order? 
I want to know when it will be shipped."""

print(f"Query: {query_2}\n")

try:
    response_2 = await agent.run(query_2, session=session)
    print("🤖 AGENT RESPONSE:")
    print(response_2)
except Exception as e:
    print(f"❌ Error: {str(e)}")

print("\n" + "="*80 + "\n")

🗣️ CUSTOMER: Turn 2
--------------------------------------------------------------------------------
Query: Can you please check the status of my recent order? 
I want to know when it will be shipped.

🤖 AGENT RESPONSE:
Hello Ramkumar,

Your order is currently in the "Pending" status. We anticipate that it will be moving to the "Shipped" status soon.

Order timing can vary depending on the processing and fulfillment stages. We will keep you updated once the status changes. Please feel free to reach out if you need further assistance or have any questions.

Thank you for your patience and understanding.

Warm regards,

[Your Company's Name] Customer Service Team




In [13]:
# Cell 10: Turn 3 - Simulate Order Shipment (Agent Updates Status)

print("🗣️ CUSTOMER: Turn 3")
print("-" * 80)

query_3 = """Great! Can you update my order status to 'Shipped' please? 
I believe the order should have been dispatched by now."""

print(f"Query: {query_3}\n")

try:
    response_3 = await agent.run(query_3, session=session)
    print("🤖 AGENT RESPONSE:")
    print(response_3)
except Exception as e:
    print(f"❌ Error: {str(e)}")

print("\n" + "="*80 + "\n")

🗣️ CUSTOMER: Turn 3
--------------------------------------------------------------------------------
Query: Great! Can you update my order status to 'Shipped' please? 
I believe the order should have been dispatched by now.

🤖 AGENT RESPONSE:
Hello Ramkumar,

Your order status has been successfully updated to "Shipped." 

**Order Details:**
- **Order ID:** 1
- **Customer Name:** Ramkumar
- **Product SKU:** LAPTOP-HP-001
- **Quantity:** 1
- **Status:** Shipped
- **Updated On:** 19th February 2026

Please feel free to reach out if you have any further questions or need additional assistance. 

Thank you for choosing our service!

Warm regards,

[Your Company's Name] Customer Service Team




In [14]:
# Cell 11: Turn 4 - Customer Reports Delivery Delay

print("🗣️ CUSTOMER: Turn 4")
print("-" * 80)

query_4 = """I need to file a complaint. My order was supposed to be delivered 2 days ago, 
but I still haven't received it. The tracking shows it's been stuck at the local 
distribution center for 3 days. This is very frustrating as I need this laptop urgently 
for work. Please escalate this issue with high priority."""

print(f"Query: {query_4}\n")

try:
    response_4 = await agent.run(query_4, session=session)
    print("🤖 AGENT RESPONSE:")
    print(response_4)
except Exception as e:
    print(f"❌ Error: {str(e)}")

print("\n" + "="*80 + "\n")

🗣️ CUSTOMER: Turn 4
--------------------------------------------------------------------------------
Query: I need to file a complaint. My order was supposed to be delivered 2 days ago, 
but I still haven't received it. The tracking shows it's been stuck at the local 
distribution center for 3 days. This is very frustrating as I need this laptop urgently 
for work. Please escalate this issue with high priority.

🤖 AGENT RESPONSE:
Hello Ramkumar,

I have escalated your complaint regarding the delivery delay with high priority. Here are the details:

**Complaint Details:**
- **Complaint ID:** 18
- **Order ID:** 1
- **Customer Name:** Ramkumar
- **Description:** Order was supposed to be delivered 2 days ago. Still stuck at the local distribution center for 3 days. Need laptop urgently for work.
- **Priority:** High
- **Remarks:** Frustrated due to delay, urgent need for work.
- **Status:** New
- **Registration Date:** 19th February 2026

Our team will review this matter promptly and work 

In [15]:
# Cell 12: Turn 5 - Agent Searches Customer's Orders and Complaints

print("🗣️ CUSTOMER: Turn 5")
print("-" * 80)

query_5 = """Can you show me a summary of all my orders and complaints? 
I want to see the complete status."""

print(f"Query: {query_5}\n")

try:
    response_5 = await agent.run(query_5, session=session)
    print("🤖 AGENT RESPONSE:")
    print(response_5)
except Exception as e:
    print(f"❌ Error: {str(e)}")

print("\n" + "="*80 + "\n")

🗣️ CUSTOMER: Turn 5
--------------------------------------------------------------------------------
Query: Can you show me a summary of all my orders and complaints? 
I want to see the complete status.

🤖 AGENT RESPONSE:
Hello Ramkumar,

Here's a summary of all your orders and complaints:

**Order Summary:**

**Order ID:** 1  
- **Product:** LAPTOP-HP-001  
- **Quantity:** 1  
- **Amount:** $2,499.00  
- **Status:** Shipped  
- **Order Date:** 19th February 2026  

---

**Complaint Summary:**

**Complaint ID:** 18  
- **Order ID:** 1  
- **Description:** Order was supposed to be delivered 2 days ago. Still stuck at the local distribution center for 3 days. Need laptop urgently for work.  
- **Priority:** High  
- **Status:** New  
- **Remarks:** Frustrated due to delay, urgent need for work.  

**Complaint ID:** 17  
- **Order ID:** ORD-10028  
- **Description:** Complaint regarding delayed order delivery.  
- **Priority:** High  
- **Status:** New  
- **Remarks:** Need to close ASAP.

In [16]:
# Cell 13: Turn 6 - Simulate Complaint Resolution

print("🗣️ CUSTOMER: Turn 6")
print("-" * 80)

query_6 = """The laptop has finally arrived! Thank you for your help. 
Can you please close the complaint and mark it as resolved?"""

print(f"Query: {query_6}\n")

try:
    response_6 = await agent.run(query_6, session=session)
    print("🤖 AGENT RESPONSE:")
    print(response_6)
except Exception as e:
    print(f"❌ Error: {str(e)}")

print("\n" + "="*80 + "\n")

🗣️ CUSTOMER: Turn 6
--------------------------------------------------------------------------------
Query: The laptop has finally arrived! Thank you for your help. 
Can you please close the complaint and mark it as resolved?

🤖 AGENT RESPONSE:
Hello Ramkumar,

I have successfully closed your complaint, marking it as resolved since the laptop has arrived.

**Complaint Resolution Details:**
- **Complaint ID:** 18
- **Status:** Resolved
- **Remarks:** Complaint closed as the laptop has arrived and the issue is resolved.
- **Updated On:** 19th February 2026

Thank you for your patience and understanding. If there's anything else you need, please don't hesitate to reach out.

Warm regards,

[Your Company's Name] Customer Service Team




In [17]:
# Cell 14: Final Summary - Verify Complete State

print("🗣️ CUSTOMER: Turn 7 - Final Verification")
print("-" * 80)

query_7 = """Perfect! Can you give me one final summary showing:
1. My order status
2. My complaint status
3. Confirmation that everything is resolved

Thank you for your excellent service!"""

print(f"Query: {query_7}\n")

try:
    response_7 = await agent.run(query_7, session=session)
    print("🤖 AGENT RESPONSE:")
    print(response_7)
except Exception as e:
    print(f"❌ Error: {str(e)}")

print("\n" + "="*80)
print("✅ CUSTOMER SERVICE SESSION COMPLETED")
print("="*80)

🗣️ CUSTOMER: Turn 7 - Final Verification
--------------------------------------------------------------------------------
Query: Perfect! Can you give me one final summary showing:
1. My order status
2. My complaint status
3. Confirmation that everything is resolved

Thank you for your excellent service!

🤖 AGENT RESPONSE:
Hello Ramkumar,

Here's a final summary of your order and complaint:

**Order Summary:**
- **Order ID:** 1
- **Product SKU:** LAPTOP-HP-001
- **Quantity:** 1
- **Amount:** $2,499.00
- **Status:** Shipped
- **Order Date:** 19th February 2026

**Complaint Summary:**
- **Complaint ID:** 18
- **Description:** Order was delayed; stuck at distribution center; needed urgently for work.
- **Priority:** High
- **Status:** Resolved
- **Remarks:** Complaint closed as the laptop arrived and issue resolved.

Everything has been successfully resolved. Thank you for your collaboration and patience along the way. 

If there's anything more you need, please feel free to contact us an

---

## 📊 Session Summary

This demonstration showcased:

### ✅ **Hybrid Tool Architecture**
- **6 Custom Python Tools** from Order Manager library
- **6 MCP Server Tools** from Complaint Management server
- Both integrated seamlessly in a single agent

### ✅ **Multi-Turn Conversation**
- **7 turns** maintaining context across the entire customer journey
- Session state preserved between interactions
- Agent remembered customer details and previous actions

### ✅ **Realistic Customer Journey**
1. Order creation (Custom Tool)
2. Status inquiry (Custom Tool)
3. Status update (Custom Tool)
4. Complaint registration (MCP Tool)
5. Comprehensive search (Both tool types)
6. Complaint resolution (MCP Tool)
7. Final verification (Both tool types)

### ✅ **Professional Service**
- Formal, courteous tone throughout
- Clear formatting and detailed responses
- Proactive information gathering
- Efficient issue resolution

### 🏗️ **Production-Ready Patterns**
- Error handling with try-catch
- Type-safe tool definitions with Pydantic
- Approval mode configured for auto-execution
- Database separation and management
- Scalable architecture supporting both patterns

---

## 🚀 Next Steps

This notebook can be extended with:
- ✨ Additional error handling middleware
- ✨ Tool call logging and analytics
- ✨ Customer satisfaction ratings
- ✨ Multi-language support
- ✨ Integration with actual payment and shipping APIs

**Congratulations!** You've successfully implemented an end-to-end customer service agent combining custom libraries and MCP servers! 🎉